In [15]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
from twilio.rest import Client 


coins = [
'BTC',
'ETH',
'ADA',
'USDT',
'DOGE',
'XRP',
'DOT',
'BCH',
'UNI',
'LTC',
'LINK',
'USDC',
'XLM',
'ETC',
'EOS',
'TRX',
'FIL',
'XMR',
'AAVE',
'XTZ',
'ATOM',
'DAI',
'KSM',
'ALGO',
'COMP',
'DASH',
'ZEC',
'WAVES',
'YFI',
'SNX',
'NANO',
'MANA',
'QTUM',
'BAT',
'GRT',
'SC',
'OMG',
'ICX',
'CRV',
'LSK',
'FLOW',
'KNC',
'EWT',
'OCEAN',
'STORJ',
'REP',
'BAL',
'REPV2',
'GNO',
'KAVA',
'ANT',
'KEEP',
'OXT',
'MLN',
'PAXG',
'TBTC',
'ETH2']


In [2]:
r = requests.get('https://api.kraken.com/0/public/Assets?currency=USD')
pr = json.loads(r.text)

In [3]:
coin_dict = {}
for coin in coins:
    resp = requests.get(f'https://api.kraken.com/0/public/OHLC?pair={coin}USD&interval=60&since=1598011000')
    try: 
        coin_dict[coin] = pd.DataFrame(json.loads(resp.text)['result'][list(json.loads(resp.text)['result'].keys())[0]],columns = ['time','open','high','low','close','avg','volume','idk'])
    except: 
        pass


In [12]:
for coin in coin_dict.keys():
    coin_dict[coin]['datetime'] = pd.to_datetime(coin_dict[coin]['time'], unit = 's')
    coin_dict[coin]['average_volume'] = coin_dict[coin]['volume'].rolling(50).mean()
    coin_dict[coin]['ma_50'] = coin_dict[coin]['close'].rolling(50).mean()
    coin_dict[coin]['volume'] = coin_dict[coin]['volume'].astype(float)
    coin_dict[coin]['open'] = coin_dict[coin]['open'].astype(float)
    coin_dict[coin]['high'] = coin_dict[coin]['high'].astype(float)
    coin_dict[coin]['low'] = coin_dict[coin]['low'].astype(float)
    coin_dict[coin]['close'] = coin_dict[coin]['close'].astype(float)
    coin_dict[coin]['3x_volume'] = coin_dict[coin]['volume'] > (coin_dict[coin]['average_volume'] *5)

In [9]:
for coin in coin_dict.keys():
    df = coin_dict[coin]
    ohlc = df[['time','open', 'high', 'low', 'close']]
    fig, ax = plt.subplots(figsize = (20,10))
    candlestick_ohlc(ax, ohlc.values, width=0.6, colorup='green', colordown='red', alpha=0.8)
    ax.plot(df[(df['3x_volume']==True)]['time'],df[(df['3x_volume']==True)]['close'],'go')
    fig.savefig(f'{coin}.jpeg')
    plt.close(fig)

In [13]:
buy_coins = []
for coin in coin_dict.keys():
    df = coin_dict[coin]
    if df.iloc[-1]['3x_volume']==True and df.iloc[-1]['open']<df.iloc[-1]['close']:
        buy_coins.append(coin)
print(buy_coins)

[]


In [ ]:
if len(buy_coins) >= 0:
    account_sid = 'AC49b7d33ba06b660a203511a81ae796cf'
    auth_token = '7b108310716502dd154df40a6fcde504'
    client = Client(account_sid, auth_token)

    message = client.messages \
                    .create(
                         body=f"Check out these coins {buy_coins}",
                         from_='+12253073889',
                         to='+17089418511'
                     )

    print(message.sid)